In [50]:
import numpy as np
import pandas as pd
from functools import partial

In [51]:
data_path = '/home/achara/data/achara/'
df = pd.read_csv(data_path + '/vp_ws_fc.csv')

In [52]:
df.head()

,DateTime,time,PC_Water_EXO,DO-Sat_Water_EXO,DO_Water_EXO,rel_fl,Chla_Water_EXO,rel_fl.1,month,year,SolarRad_Air_LiCor
0,10-1-2017,0:01,0.1448,101.82,9.14,0.844,0.800312,0.844,10,2017,127.38941
1,10-1-2017,0:03,0.1340,101.74,9.13,0.826,0.553970,0.826,10,2017,127.38941
2,10-1-2017,0:04,0.1232,101.67,9.13,0.826,0.553919,0.826,10,2017,127.38941
3,10-1-2017,0:05,0.1124,101.57,9.12,0.826,0.437228,0.826,10,2017,127.38941
4,10-1-2017,0:07,0.1160,101.50,9.11,0.844,0.489017,0.844,10,2017,127.38941


In [53]:
nut_df = pd.read_csv(data_path + 'SINC_input_data_EDI.csv')

In [54]:
def convert_date_format(date: str) -> str:
    parts = date.split('-')
    parts.append(parts.pop(0))
    standardize = list(map(int, parts))
    return '-'.join(list(map(str, standardize)))

In [55]:
nut_df.Datetime = nut_df.Datetime.map(lambda x: x.split(' ')[0]).map(convert_date_format)

In [56]:
keys = ['Datetime','NO3', 'NH4', 'PP']
nut_df_filtered = nut_df.loc[:, keys]

In [57]:
nut_df_filtered

,Datetime,NO3,NH4,PP
0,1-22-2018,0.552,0.0050,2.1741
1,2-28-2018,0.362,0.0185,3.8954
2,3-20-2018,0.587,0.0050,9.7426
3,4-4-2018,0.248,0.0050,7.5395
4,4-4-2018,0.195,0.0050,25.6634
...,...,...,...,...
675,10-23-2019,0.825,0.0050,13.0438
676,10-23-2019,0.732,0.0050,8.8632
677,10-23-2019,0.851,0.0050,6.5220
678,11-6-2019,0.905,0.0134,2.8660


In [58]:
def get_part(idx: int, date: str) -> str:
    return date.split('-')[idx]

In [59]:
nut_df_filtered['month'] = nut_df_filtered.Datetime.map(partial(get_part, 0)).map(int)
nut_df_filtered['day'] = nut_df_filtered.Datetime.map(partial(get_part, 1)).map(int)
nut_df_filtered['year'] = nut_df_filtered.Datetime.map(partial(get_part, 2)).map(int)

In [60]:
nut_df_filtered = nut_df_filtered.loc[:,  ~nut_df_filtered.columns.isin(['Datetime','day'])]

In [61]:
nut_df_mean = nut_df_filtered.groupby(['year','month'], as_index=False).mean()

In [62]:
merged = df.merge(nut_df_mean, on=['year','month' ], how='left')

In [63]:
merged.isna().sum()

DateTime              0
time                  0
PC_Water_EXO          0
DO-Sat_Water_EXO      0
DO_Water_EXO          0
rel_fl                0
Chla_Water_EXO        0
rel_fl.1              0
month                 0
year                  0
SolarRad_Air_LiCor    0
NO3                   0
NH4                   0
PP                    0
dtype: int64

In [64]:
merged.columns

Index(['DateTime', 'time', 'PC_Water_EXO', 'DO-Sat_Water_EXO', 'DO_Water_EXO',
       'rel_fl', 'Chla_Water_EXO', 'rel_fl.1', 'month', 'year',
       'SolarRad_Air_LiCor', 'NO3', 'NH4', 'PP'],
      dtype='object')

In [65]:
merged['day']=merged.DateTime.map(partial(get_part, 1)).map(int)

In [66]:
merged.head()

,DateTime,time,PC_Water_EXO,DO-Sat_Water_EXO,DO_Water_EXO,rel_fl,Chla_Water_EXO,rel_fl.1,month,year,SolarRad_Air_LiCor,NO3,NH4,PP,day
0,10-1-2017,0:01,0.1448,101.82,9.14,0.844,0.800312,0.844,10,2017,127.38941,0.191,0.005,16.681741,1
1,10-1-2017,0:03,0.1340,101.74,9.13,0.826,0.553970,0.826,10,2017,127.38941,0.191,0.005,16.681741,1
2,10-1-2017,0:04,0.1232,101.67,9.13,0.826,0.553919,0.826,10,2017,127.38941,0.191,0.005,16.681741,1
3,10-1-2017,0:05,0.1124,101.57,9.12,0.826,0.437228,0.826,10,2017,127.38941,0.191,0.005,16.681741,1
4,10-1-2017,0:07,0.1160,101.50,9.11,0.844,0.489017,0.844,10,2017,127.38941,0.191,0.005,16.681741,1


In [67]:
lst = pd.read_csv(data_path+'LST.csv')

In [68]:
lst.rename(columns={'YEAR':'year', 'MO':'month','DY':'day'}, inplace=True)

In [69]:
final_df = merged.merge(lst, on=['year','month','day'], how='left')

In [70]:
mean_df = final_df.groupby(['month'])['PRECTOTCORR', 'ALLSKY_SFC_LW_DWN', 'ALLSKY_SFC_SW_DWN'].mean()

/home/achara/achara_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [71]:
mean_df

,PRECTOTCORR,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN
month,,,
5,2.628489,334.779173,5.391961
6,3.492326,344.496395,5.933266
7,2.637167,367.263344,6.410781
8,3.271875,358.454086,5.399513
9,2.670990,341.080583,4.167955
10,4.503493,311.677335,2.355409
11,2.941629,275.013789,1.673628
12,2.209290,278.067458,1.218532


In [72]:
final_df['PRECTOTCORR'] = final_df['PRECTOTCORR'].fillna(
    final_df.month.map(lambda m: mean_df.at[m, 'PRECTOTCORR']))

In [73]:
final_df['ALLSKY_SFC_LW_DWN'] = final_df['ALLSKY_SFC_LW_DWN'].fillna(
    final_df.month.map(lambda m: mean_df.at[m, 'ALLSKY_SFC_LW_DWN']))

In [74]:
final_df['ALLSKY_SFC_SW_DWN'] = final_df['ALLSKY_SFC_SW_DWN'].fillna(
    final_df.month.map(lambda m: mean_df.at[m, 'ALLSKY_SFC_SW_DWN']))

In [75]:
final_df.isna().sum()

DateTime              0
time                  0
PC_Water_EXO          0
DO-Sat_Water_EXO      0
DO_Water_EXO          0
rel_fl                0
Chla_Water_EXO        0
rel_fl.1              0
month                 0
year                  0
SolarRad_Air_LiCor    0
NO3                   0
NH4                   0
PP                    0
day                   0
PRECTOTCORR           0
ALLSKY_SFC_SW_DWN     0
ALLSKY_SFC_LW_DWN     0
dtype: int64

In [76]:
final_df_filtered = final_df.loc[:, ~final_df.columns.isin(['month','year','day'])]

In [77]:
final_df_filtered.head()

,DateTime,time,PC_Water_EXO,DO-Sat_Water_EXO,DO_Water_EXO,rel_fl,Chla_Water_EXO,rel_fl.1,SolarRad_Air_LiCor,NO3,NH4,PP,PRECTOTCORR,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_LW_DWN
0,10-1-2017,0:01,0.1448,101.82,9.14,0.844,0.800312,0.844,127.38941,0.191,0.005,16.681741,4.503493,2.355409,311.677335
1,10-1-2017,0:03,0.1340,101.74,9.13,0.826,0.553970,0.826,127.38941,0.191,0.005,16.681741,4.503493,2.355409,311.677335
2,10-1-2017,0:04,0.1232,101.67,9.13,0.826,0.553919,0.826,127.38941,0.191,0.005,16.681741,4.503493,2.355409,311.677335
3,10-1-2017,0:05,0.1124,101.57,9.12,0.826,0.437228,0.826,127.38941,0.191,0.005,16.681741,4.503493,2.355409,311.677335
4,10-1-2017,0:07,0.1160,101.50,9.11,0.844,0.489017,0.844,127.38941,0.191,0.005,16.681741,4.503493,2.355409,311.677335


In [78]:
final_df.to_csv(data_path+'/forecast_input.csv', index=False)